# EDA on Retail Gold Schema Data

**Overview:**  
This notebook is focused on performing **Exploratory Data Analysis (EDA)** on the `gold` schema of our dataset. The schema contains both fact and dimension tables that represent sales, stock, customers, staff, products, and stores.  

**Schema Summary:**  
- **Fact Tables:**  
    - `fact_sales`: Contains transactional sales data with details such as quantity, price, discount, and dates.  
    - `fact_stocks`: Contains inventory data for stores and products.  

- **Dimension Tables:**  
    - `dim_customers`: Customer information including name, contact, and address.  
    - `dim_staffs`: Staff details including name, store association, and manager.  
    - `dim_products`: Product catalog with category, brand, pricing, and model year.  
    - `dim_stores`: Store details including address and contact information.  

**Objectives:**  
1. Connect to SQL Server and load the gold schema data into Python using **pyodbc**.  
2. Explore the data to gain **insights and trends**.
3. Perform **summary statistics and visualizations** to understand distributions, relationships, and patterns.  
4. Create charts and plots to uncover interesting aspects of the business, such as sales per product, store performance, or customer segmentation.  
5. Prepare the data for further **analysis and reporting in Power BI**.  

**Tools & Libraries:**  
- Python (`pandas`, `numpy`)  
- SQL connection (`pyodbc`)  
- Data visualization (`matplotlib`, `seaborn`)  

This notebook will help generate insights from the gold dataset and create a foundation for **Power BI dashboards and reporting**.


# Loading Gold Schema Data into Python

**Overview:**  
We will load data from our SQL Server `gold` schema into Python using `pyodbc`. This will allow us to perform EDA, generate insights, and visualize the data.

**Notes:**  
- All tables (fact and dimension tables) will be loaded as Pandas DataFrames.  
- These DataFrames will be used for charts, aggregations, and other exploratory tasks.  

In [ ]:
import pyodbc
import pandas as pd

# Connection parameters
server = 'your_server_name\SQLEXPRESS'        
database = 'DataWarehouse'           


# Connection string
conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\puroh\AppData\Local\Temp\ipykernel_2680\342284136.py:5: SyntaxWarning: invalid escape sequence '\S'
  server = 'KRUSHNA\SQLEXPRESS'


In [2]:
# Connect
conn = pyodbc.connect(conn_str)

In [ ]:
# Load all the tables
df_customers = pd.read_sql_query('SELECT * FROM gold.dim_customers', conn)
df_products  = pd.read_sql_query('SELECT * FROM gold.dim_products', conn)
df_staffs    = pd.read_sql_query('SELECT * FROM gold.dim_staffs', conn)
df_stores    = pd.read_sql_query('SELECT * FROM gold.dim_stores', conn)
df_sales     = pd.read_sql_query('SELECT * FROM gold.fact_sales', conn)
df_stocks    = pd.read_sql_query('SELECT * FROM gold.fact_stocks', conn)

In [4]:
# Printing rows and columns of each DataFrame
print(f"dim_customers: {df_customers.shape}")
print(f"dim_products:  {df_products.shape}")
print(f"dim_staffs:    {df_staffs.shape}")
print(f"dim_stores:    {df_stores.shape}")
print(f"fact_sales:    {df_sales.shape}")
print(f"fact_stocks:   {df_stocks.shape}")

dim_customers: (1445, 10)
dim_products:  (321, 8)
dim_staffs:    (10, 8)
dim_stores:    (3, 9)
fact_sales:    (4722, 19)
fact_stocks:   (939, 4)


# Overview of all dataframes

- .shape → rows, columns
- .info() → data types, nulls
- .head() → first few rows
- .describe() → stats for numeric columns

In [5]:
dfs = {
    "Customers": df_customers,
    "Products": df_products,
    "Sales": df_sales,
    "Staffs": df_staffs,
    "Stores": df_stores,
    "Stocks": df_stocks
}

for name, df in dfs.items():
    print(f"\n=== {name} ===")
    print("Shape:", df.shape)
    print()
    print(df.info())
    print()
    print("Top 3 rows :")
    print(df.head(3))
    print()
    print("Describe")
    print(df.describe())




=== Customers ===
Shape: (1445, 10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   1445 non-null   int64 
 1   first_name    1445 non-null   object
 2   last_name     1445 non-null   object
 3   phone         178 non-null    object
 4   email         1445 non-null   object
 5   street        1445 non-null   object
 6   city          1445 non-null   object
 7   state         1445 non-null   object
 8   zip_code      1445 non-null   object
 9   full_address  1445 non-null   object
dtypes: int64(1), object(9)
memory usage: 113.0+ KB
None

Top 3 rows :
   customer_id first_name last_name phone                  email  \
0            1      Debra     Burks  None  debra.burks@yahoo.com   
1            2      Kasha      Todd  None   kasha.todd@yahoo.com   
2            3     Tameka    Fisher  None  tameka.fisher@aol.com   

       

## Dataset Overview – Initial Observations  

### Customers (1,445 rows, 10 columns)  
- Majority of fields are complete.  
- **Phone numbers** missing for most customers (~88% missing).  

### Products (321 rows, 8 columns)  
- Covers **7 categories** and multiple brands.  
- Price ranges from **$89.99 to $11,999.99**, large spread.  

### Sales (4,722 rows, 19 columns)  
- Rich dataset with customer, product, staff, store, and order details.  
- Dates (`order_date`, `required_date`, `shipped_date`) mostly valid, but **~500 missing shipped dates**.  

### Staffs (10 rows, 8 columns)  
- Small team of 10 staff members.  
- All marked as **active**.  
- Includes hierarchy via `manager_id`.  

### Stores (3 rows, 9 columns)  
- Only 3 stores in the dataset.  
- Each store has complete contact and address details.  

### Stocks (939 rows, 4 columns)  
- Tracks inventory levels per store per product.  
- Some products have `quantity = 0` (flagged as `out_of_stock = True`).  
- Quantities range from 0 to 30 units.  
